<a href="https://colab.research.google.com/github/nahilsobh/BBox-Label-Tool/blob/master/Colab5_for_deeplearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's start with showing all the code from the previous lesson. Run this to train the model. Take note of the accuracy on the training and validation sets.

In [1]:
# ----------------
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
      -O /tmp/cats_and_dogs_filtered.zip
  
# ----------------
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref   = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# ----------------
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir      = os.path.join( base_dir, 'train'     )
validation_dir = os.path.join( base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join( train_dir, 'cats'     )
train_dogs_dir = os.path.join( train_dir, 'dogs'     )

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join( validation_dir, 'cats' )
validation_dogs_dir = os.path.join( validation_dir, 'dogs' )

# ----------------
import tensorflow                           as     tf
from   tensorflow.keras.optimizers          import RMSprop
from   tensorflow.keras.preprocessing.image import ImageDataGenerator

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    

    tf.keras.layers.Conv2D      ( 16, (3,3), activation='relu', input_shape=(150, 150, 3)), # First convolution
    tf.keras.layers.MaxPooling2D      (2,2)                       , # First  max pooling 
    
    tf.keras.layers.Conv2D      ( 32, (3,3), activation='relu'   ), # Second convoution
    tf.keras.layers.MaxPooling2D      (2,2),                        # Second max pooling 
    
    tf.keras.layers.Conv2D      ( 64, (3,3), activation='relu'   ), # Third  convolution
    tf.keras.layers.MaxPooling2D      (2,2)                       , # Third  max pooling
    
    tf.keras.layers.Flatten     ()                                , # Flatten the results to feed into a DNN
    
    tf.keras.layers.Dense       (512,        activation='relu'   ), # 512 neuron hidden layer
    
    tf.keras.layers.Dense       (  1,        activation='sigmoid')  #Only 1 output neuron. 
                                                                   # It will contain a value from 0-1 
                                                                   # where 0 for 1 class ('cats') 
                                                                   # and 1 for the other ('dogs')
])

# ----------------
model.compile( optimizer =   RMSprop(lr=0.001)    ,
                    loss =  'binary_crossentropy' ,
                 metrics = ['acc']
             )

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator      = train_datagen.flow_from_directory( train_dir               , # directory for training images
                                                          batch_size  = 20        ,
                                                          class_mode  = 'binary'  , # binary labels to use with binary_crossentropy loss
                                                          target_size = (150, 150)  # All images are resized to 150x150
                                                         
                                                        )     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory( validation_dir          ,
                                                          batch_size  = 20        ,
                                                          class_mode  = 'binary'  , 
                                                          target_size = (150, 150),
                                                        )

# --------------------
history = model.fit_generator( train_generator                         ,
                               validation_data  = validation_generator ,
                               steps_per_epoch  = 100                  ,  # 2000 images = batch_size * steps
                               epochs           =  15                  ,
                               validation_steps =  50                  ,  # 1000 images = batch_size * steps
                               verbose          =   2                     # Verbosity mode (one line per epoch)
                             )

--2019-02-09 03:53:29--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  91.7MB/s    in 0.7s    

2019-02-09 03:53:30 (91.7 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Instructions for updating:
Colocations handled automatically by placer.
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
50/50 [==============================] - 27s 539ms/step - loss: 0.6947 - acc: 0.5690
 - 162s - loss: 1.2301 - acc: 0.5620 - val_loss: 0.6947 - val_acc: 0.5690
Epoch 2/15
50/50 [====

The Training Accuracy is about 98%, and the validation accuracy about 70%. This is a great example of overfitting -- which in short means that it can do very well with images it has seen before, but not so well with images it hasn't. Let's see if we can do better to avoid overfitting -- and one simple method is to augment the images a bit. If you think about it, most pictures of a cat are very similar -- the ears are at the top, then the eyes, then the mouth etc. Things like the distance between the eyes and ears will always be quite similar too. 

What if we tweak with the images to change this up a bit -- rotate the image, squash it, etc.  That's what image augementation is all about. And there's an API that makes it easy...

Now take a look at the ImageGenerator. There are properties on it that you can use to augment the image. 

```
# Updated to do image augmentation
train_datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
```
These are just a few of the options available (for more, see the Keras documentation. Let's quickly go over what we just wrote:

* rotation_range is a value in degrees (0–180), a range within which to randomly rotate pictures.
* width_shift and height_shift are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally.
* shear_range is for randomly applying shearing transformations.
* zoom_range is for randomly zooming inside pictures.
* horizontal_flip is for randomly flipping half of the images horizontally. This is relevant when there are no assumptions of horizontal assymmetry (e.g. real-world pictures).
* fill_mode is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.


In the full listing below, find the ImageDataGenerator for the training data and replace it with one that uses Image Augmentation like above.
Rerun the training...what's the impact? Don't forget to keep the normalization code. It's not in the above snippet, so add it as a parameter! :)


In [2]:
# ----------------
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
      -O /tmp/cats_and_dogs_filtered.zip
  
# ----------------
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref   = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# ----------------
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir      = os.path.join( base_dir, 'train'     )
validation_dir = os.path.join( base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join( train_dir, 'cats'     )
train_dogs_dir = os.path.join( train_dir, 'dogs'     )

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join( validation_dir, 'cats' )
validation_dogs_dir = os.path.join( validation_dir, 'dogs' )

# ----------------
import tensorflow                           as     tf
from   tensorflow.keras.optimizers          import RMSprop
from   tensorflow.keras.preprocessing.image import ImageDataGenerator

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    

    tf.keras.layers.Conv2D      ( 16, (3,3), activation='relu', input_shape=(150, 150, 3)), # First convolution
    tf.keras.layers.MaxPooling2D      (2,2)                       , # First  max pooling 
    
    tf.keras.layers.Conv2D      ( 32, (3,3), activation='relu'   ), # Second convoution
    tf.keras.layers.MaxPooling2D      (2,2),                        # Second max pooling 
    
    tf.keras.layers.Conv2D      ( 64, (3,3), activation='relu'   ), # Third  convolution
    tf.keras.layers.MaxPooling2D      (2,2)                       , # Third  max pooling
    
    tf.keras.layers.Flatten     ()                                , # Flatten the results to feed into a DNN
    
    tf.keras.layers.Dense       (512,        activation='relu'   ), # 512 neuron hidden layer
    
    tf.keras.layers.Dense       (  1,        activation='sigmoid')  #Only 1 output neuron. 
                                                                   # It will contain a value from 0-1 
                                                                   # where 0 for 1 class ('cats') 
                                                                   # and 1 for the other ('dogs')
])

# ----------------
model.compile( optimizer =   RMSprop(lr=0.001)    ,
                    loss =  'binary_crossentropy' ,
                 metrics = ['acc']
             )

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator      = train_datagen.flow_from_directory( train_dir               , # directory for training images
                                                          batch_size  = 20        ,
                                                          class_mode  = 'binary'  , # binary labels to use with binary_crossentropy loss
                                                          target_size = (150, 150)  # All images are resized to 150x150
                                                         
                                                        )     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory( validation_dir          ,
                                                          batch_size  = 20        ,
                                                          class_mode  = 'binary'  , 
                                                          target_size = (150, 150),
                                                        )

# --------------------
history = model.fit_generator( train_generator                         ,
                               validation_data  = validation_generator ,
                               steps_per_epoch  = 100                  ,  # 2000 images = batch_size * steps
                               epochs           =  15                  ,
                               validation_steps =  50                  ,  # 1000 images = batch_size * steps
                               verbose          =   2                     # Verbosity mode (one line per epoch)
                             )

--2019-02-09 06:12:42--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   106MB/s    in 0.6s    

2019-02-09 06:12:43 (106 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
50/50 [==============================] - 14s 279ms/step - loss: 0.6188 - acc: 0.6750
 - 81s - loss: 0.7731 - acc: 0.5860 - val_loss: 0.6188 - val_acc: 0.6750


You'll probably see something like accuracy on training of around 72% (.7195) and accuracy on validation of around 74% (.7350) Your values may differ due to the random nature of the augmentation.

It also likely slowed down the training as all the image processing was also being done. 

Notice how the latter has likely gone up? That's because, thanks to all of the augmentation of images that was going on, a greater diversity of shapes and sizes was 'seen' by the model in the training process. This becomes a handy technique to avoid overfitting. Try it out, and experiment with different values for each of the parameters to see what the impact on the results might be.


TODO

Now let's talk dropouts and
tf.keras.layers.Dropout(0.5),

In [3]:
# ----------------
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
      -O /tmp/cats_and_dogs_filtered.zip
  
# ----------------
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref   = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# ----------------
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir      = os.path.join( base_dir, 'train'     )
validation_dir = os.path.join( base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join( train_dir, 'cats'     )
train_dogs_dir = os.path.join( train_dir, 'dogs'     )

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join( validation_dir, 'cats' )
validation_dogs_dir = os.path.join( validation_dir, 'dogs' )

# ----------------
import tensorflow                           as     tf
from   tensorflow.keras.optimizers          import RMSprop
from   tensorflow.keras.preprocessing.image import ImageDataGenerator

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    

    tf.keras.layers.Conv2D      ( 16, (3,3), activation='relu', input_shape=(150, 150, 3)), # First convolution
    tf.keras.layers.MaxPooling2D      (2,2)                       , # First  max pooling 
    
    tf.keras.layers.Conv2D      ( 32, (3,3), activation='relu'   ), # Second convoution
    tf.keras.layers.MaxPooling2D      (2,2),                        # Second max pooling 
    
    tf.keras.layers.Conv2D      ( 64, (3,3), activation='relu'   ), # Third  convolution
    tf.keras.layers.MaxPooling2D      (2,2)                       , # Third  max pooling
    
    tf.keras.layers.Flatten     ()                                , # Flatten the results to feed into a DNN
    
    tf.keras.layers.Dense       (512,        activation='relu'   ), # 512 neuron hidden layer
    
    tf.keras.layers.Dense       (  1,        activation='sigmoid')  #Only 1 output neuron. 
                                                                   # It will contain a value from 0-1 
                                                                   # where 0 for 1 class ('cats') 
                                                                   # and 1 for the other ('dogs')
])

# ----------------
model.compile( optimizer =   RMSprop(lr=0.001)    ,
                    loss =  'binary_crossentropy' ,
                 metrics = ['acc']
             )

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator      = train_datagen.flow_from_directory( train_dir               , # directory for training images
                                                          batch_size  = 20        ,
                                                          class_mode  = 'binary'  , # binary labels to use with binary_crossentropy loss
                                                          target_size = (150, 150)  # All images are resized to 150x150
                                                         
                                                        )     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory( validation_dir          ,
                                                          batch_size  = 20        ,
                                                          class_mode  = 'binary'  , 
                                                          target_size = (150, 150),
                                                        )

# --------------------
history = model.fit_generator( train_generator                         ,
                               validation_data  = validation_generator ,
                               steps_per_epoch  = 100                  ,  # 2000 images = batch_size * steps
                               epochs           =  15                  ,
                               validation_steps =  50                  ,  # 1000 images = batch_size * steps
                               verbose          =   2                     # Verbosity mode (one line per epoch)
                             )

--2019-02-09 06:14:29--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   223MB/s    in 0.3s    

2019-02-09 06:14:30 (223 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
50/50 [==============================] - 14s 289ms/step - loss: 0.6678 - acc: 0.6260
 - 83s - loss: 0.8189 - acc: 0.5435 - val_loss: 0.6678 - val_acc: 0.6260
